In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

nclasses = 43 # GTSRB as 43 classes

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 1, kernel_size=1)
        self.conv2 = nn.Conv2d(1, 29, kernel_size=5)
        self.maxpool2 = nn.MaxPool2d(3, stride=2 , ceil_mode=True)
        self.conv3 = nn.Conv2d(29, 59, kernel_size=3)
        self.maxpool3 = nn.MaxPool2d(3, stride=2 , ceil_mode=True)
        self.conv4 = nn.Conv2d(59, 74, kernel_size=3)
        self.maxpool4 = nn.MaxPool2d(3, stride=2 , ceil_mode=True)
        self.conv2_drop = nn.Dropout2d()
        self.conv3_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(1184, 300)
        self.fc2 = nn.Linear(300, nclasses)
        self.conv0_bn = nn.BatchNorm2d(3)
        self.conv1_bn = nn.BatchNorm2d(1)
        self.conv2_bn = nn.BatchNorm2d(29)
        self.conv3_bn = nn.BatchNorm2d(59)
        self.conv4_bn = nn.BatchNorm2d(74)
        self.dense1_bn = nn.BatchNorm1d(300)
    def forward(self, x):
        x =  F.relu(self.conv1_bn(self.conv1(self.conv0_bn(x))))
        x = F.relu(self.conv2_bn(self.conv2(x)))
        x = F.relu(self.conv3_bn(self.conv3( self.maxpool2(x))))
        x = F.relu(self.conv4_bn(self.conv4( self.maxpool3(x))))
        x = self.maxpool4(x)        
        x = x.view(-1, 1184)
        x = F.relu(self.fc1(x))
        x = self.dense1_bn(x)
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        x = F.dropout(x, training=self.training)
        return F.log_softmax(x)
        


In [2]:
from __future__ import print_function
import argparse
import os
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import cv2

batch_size =50                                                                                                                                                                                     
epochs = 10000                                                                                                                                                                                                                                                                                                                                                                                 
seed = 1                                                                                                                                                                                           
log_interval=180                                                                                                                                                                                   
data = "DatasetAug"                                                                                                                                                                                                                                                                                                                                                                                          
torch.manual_seed(1)                                                                                                                                                                               
lr =0.007                                                                                                                                                                                          
momentum = 0.8                                                                                                                                                                                     
decay = 0.9                                                                                                                                                                                        
step = 1000                                                                                                                                                                                        
l2_norm = 0.00001  
cuda = True
resume = False
# These may change as described in paper

In [3]:
class ADataset(Dataset):
    def __init__(self, images_filepaths, transform=None):
        self.images_filepaths = images_filepaths
        self.transform = transform

    def __len__(self):
        return len(self.images_filepaths)

    def __getitem__(self, idx):
        image_filepath = self.images_filepaths[idx]
        image = cv2.imread(image_filepath)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = int(os.path.normpath(image_filepath).split(os.sep)[-2])
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        return image, label

In [5]:
from data import initialize_data, data_transforms 
initialize_data("DatasetAug") # extracts the zip files, makes a validation set

# train_loader = DataLoader(
#     datasets.ImageFolder(data + '/train_images',
#                          transform=data_transforms),
#     batch_size=batch_size, shuffle=True, num_workers=1)


def generateImgFilePaths(base_path):
    imgpths = []
    for i in range(nclasses):
        fldr=("/0000" if i<10 else "/000")+str(i)
        for path in os.listdir(base_path+fldr):
            full_path = os.path.join(base_path+fldr, path)
            if os.path.isfile(full_path) and not full_path.endswith(".csv"):
                imgpths.append(full_path)
    return imgpths

print(len(generateImgFilePaths(data + '/train_images')))

fxc = ADataset(images_filepaths=generateImgFilePaths(data + '/train_images'), transform=data_transforms)
train_loader = DataLoader(fxc, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

fxv = ADataset(images_filepaths=generateImgFilePaths(data + '/val_images'), transform=data_transforms)
val_loader = DataLoader(fxv, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

# val_loader = DataLoader(
#     datasets.ImageFolder(data + '/val_images',
#                          transform=data_transforms),
#     batch_size=batch_size, shuffle=False, num_workers=1)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = Net()
if  cuda: 
    model.to(device)

if resume :
    state_dict = torch.load("model_78.pth")
    model.load_state_dict(state_dict) 



DatasetAug/val_images
DatasetAug/val_images not found, making a validation set
39519
cuda:0


In [5]:
def validation():
    model.eval()
    validation_loss = 0
    correct = 0
    for data, target in val_loader:
        data, target = Variable(data), Variable(target)
        data=data.float()
        data =data.to(device)
        target =target.to(device)
        output = model(data)
        validation_loss += F.nll_loss(output, target, size_average=False).data.item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    validation_loss /= len(val_loader.dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        validation_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))
    return validation_loss


def train(epoch , train_loader):
#     model.float()
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        data=data.float()
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target).cuda()
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset),100. * batch_idx / len(train_loader), loss.data.item()))

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=lr , momentum=momentum, weight_decay=l2_norm, nesterov=True)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=decay)

temp = 10
for epoch in range(1, epochs):
    train(epoch, train_loader)
    val = validation()
    if epoch % step :
        scheduler.step()
    if val < temp : 
        temp = val
        model_file = 'model_' + str(epoch) + '.pth'
        torch.save(model.state_dict(), model_file)
        print('\nSaved model to ' + model_file + '. You can run `python evaluate.py ' + model_file + '` to generate the Kaggle formatted csv file')

<ipython-input-1-493c3bbaedf2>:39: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [0/35339 (0%)]	Loss: 4.791858
Train Epoch: 1 [9000/35339 (25%)]	Loss: 3.335840
Train Epoch: 1 [18000/35339 (51%)]	Loss: 2.782558
Train Epoch: 1 [27000/35339 (76%)]	Loss: 2.660691


c:\users\utkarsh\appdata\local\programs\python\python38\lib\site-packages\torch\nn\_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Validation set: Average loss: 2.6919, Accuracy: 1465/3870 (38%)


Saved model to model_1.pth. You can run `python evaluate.py model_1.pth` to generate the Kaggle formatted csv file
Train Epoch: 2 [0/35339 (0%)]	Loss: 2.790171
Train Epoch: 2 [9000/35339 (25%)]	Loss: 2.440998
Train Epoch: 2 [18000/35339 (51%)]	Loss: 2.350769
Train Epoch: 2 [27000/35339 (76%)]	Loss: 2.349819

Validation set: Average loss: 2.0276, Accuracy: 2349/3870 (61%)


Saved model to model_2.pth. You can run `python evaluate.py model_2.pth` to generate the Kaggle formatted csv file
Train Epoch: 3 [0/35339 (0%)]	Loss: 2.079546
Train Epoch: 3 [9000/35339 (25%)]	Loss: 2.317107
Train Epoch: 3 [18000/35339 (51%)]	Loss: 2.041207
Train Epoch: 3 [27000/35339 (76%)]	Loss: 2.085842

Validation set: Average loss: 1.4350, Accuracy: 2951/3870 (76%)


Saved model to model_3.pth. You can run `python evaluate.py model_3.pth` to generate the Kaggle formatted csv file
Train Epoch: 4 [0/35339 (0%)]	Loss: 1.998011
Train Epoch: 4 [9000/

Train Epoch: 25 [27000/35339 (76%)]	Loss: 1.674081

Validation set: Average loss: 0.5709, Accuracy: 3473/3870 (90%)


Saved model to model_25.pth. You can run `python evaluate.py model_25.pth` to generate the Kaggle formatted csv file
Train Epoch: 26 [0/35339 (0%)]	Loss: 1.871736
Train Epoch: 26 [9000/35339 (25%)]	Loss: 1.756385
Train Epoch: 26 [18000/35339 (51%)]	Loss: 1.968634
Train Epoch: 26 [27000/35339 (76%)]	Loss: 2.162525

Validation set: Average loss: 0.5573, Accuracy: 3466/3870 (90%)


Saved model to model_26.pth. You can run `python evaluate.py model_26.pth` to generate the Kaggle formatted csv file
Train Epoch: 27 [0/35339 (0%)]	Loss: 2.094916
Train Epoch: 27 [9000/35339 (25%)]	Loss: 1.751043
Train Epoch: 27 [18000/35339 (51%)]	Loss: 1.842485
Train Epoch: 27 [27000/35339 (76%)]	Loss: 1.721963

Validation set: Average loss: 0.5724, Accuracy: 3461/3870 (89%)

Train Epoch: 28 [0/35339 (0%)]	Loss: 1.679042
Train Epoch: 28 [9000/35339 (25%)]	Loss: 1.491780
Train Epoch: 28 [18000/

Train Epoch: 54 [9000/35339 (25%)]	Loss: 1.884130
Train Epoch: 54 [18000/35339 (51%)]	Loss: 1.480478
Train Epoch: 54 [27000/35339 (76%)]	Loss: 1.816823

Validation set: Average loss: 0.5400, Accuracy: 3467/3870 (90%)

Train Epoch: 55 [0/35339 (0%)]	Loss: 1.917657
Train Epoch: 55 [9000/35339 (25%)]	Loss: 1.417148
Train Epoch: 55 [18000/35339 (51%)]	Loss: 2.109292
Train Epoch: 55 [27000/35339 (76%)]	Loss: 1.370273

Validation set: Average loss: 0.5507, Accuracy: 3464/3870 (90%)

Train Epoch: 56 [0/35339 (0%)]	Loss: 1.687401
Train Epoch: 56 [9000/35339 (25%)]	Loss: 1.835034
Train Epoch: 56 [18000/35339 (51%)]	Loss: 1.956649
Train Epoch: 56 [27000/35339 (76%)]	Loss: 1.597691

Validation set: Average loss: 0.5339, Accuracy: 3473/3870 (90%)

Train Epoch: 57 [0/35339 (0%)]	Loss: 1.592360
Train Epoch: 57 [9000/35339 (25%)]	Loss: 1.838902
Train Epoch: 57 [18000/35339 (51%)]	Loss: 1.756891
Train Epoch: 57 [27000/35339 (76%)]	Loss: 2.004721

Validation set: Average loss: 0.5508, Accuracy: 3457/38

Train Epoch: 85 [9000/35339 (25%)]	Loss: 1.764893
Train Epoch: 85 [18000/35339 (51%)]	Loss: 1.771537
Train Epoch: 85 [27000/35339 (76%)]	Loss: 1.960514

Validation set: Average loss: 0.5140, Accuracy: 3499/3870 (90%)

Train Epoch: 86 [0/35339 (0%)]	Loss: 2.110533
Train Epoch: 86 [9000/35339 (25%)]	Loss: 1.439213
Train Epoch: 86 [18000/35339 (51%)]	Loss: 1.447579
Train Epoch: 86 [27000/35339 (76%)]	Loss: 1.909320

Validation set: Average loss: 0.5113, Accuracy: 3491/3870 (90%)

Train Epoch: 87 [0/35339 (0%)]	Loss: 1.951980
Train Epoch: 87 [9000/35339 (25%)]	Loss: 1.677148
Train Epoch: 87 [18000/35339 (51%)]	Loss: 1.676214
Train Epoch: 87 [27000/35339 (76%)]	Loss: 1.983112

Validation set: Average loss: 0.5384, Accuracy: 3486/3870 (90%)

Train Epoch: 88 [0/35339 (0%)]	Loss: 1.051405
Train Epoch: 88 [9000/35339 (25%)]	Loss: 1.811880
Train Epoch: 88 [18000/35339 (51%)]	Loss: 1.745883
Train Epoch: 88 [27000/35339 (76%)]	Loss: 1.680368

Validation set: Average loss: 0.5359, Accuracy: 3492/38

Train Epoch: 115 [27000/35339 (76%)]	Loss: 1.993414

Validation set: Average loss: 0.5486, Accuracy: 3466/3870 (90%)

Train Epoch: 116 [0/35339 (0%)]	Loss: 1.738807
Train Epoch: 116 [9000/35339 (25%)]	Loss: 1.539240
Train Epoch: 116 [18000/35339 (51%)]	Loss: 1.545886
Train Epoch: 116 [27000/35339 (76%)]	Loss: 1.723542

Validation set: Average loss: 0.5177, Accuracy: 3512/3870 (91%)

Train Epoch: 117 [0/35339 (0%)]	Loss: 2.018647
Train Epoch: 117 [9000/35339 (25%)]	Loss: 1.792963
Train Epoch: 117 [18000/35339 (51%)]	Loss: 1.983714
Train Epoch: 117 [27000/35339 (76%)]	Loss: 1.623772

Validation set: Average loss: 0.5071, Accuracy: 3509/3870 (91%)

Train Epoch: 118 [0/35339 (0%)]	Loss: 2.004623
Train Epoch: 118 [9000/35339 (25%)]	Loss: 1.663182
Train Epoch: 118 [18000/35339 (51%)]	Loss: 1.996597
Train Epoch: 118 [27000/35339 (76%)]	Loss: 1.552620

Validation set: Average loss: 0.5462, Accuracy: 3472/3870 (90%)

Train Epoch: 119 [0/35339 (0%)]	Loss: 1.478423
Train Epoch: 119 [9000/35339 (2